In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHG6_Signal_280Deg_07152017.txt', 'SHG6_Background_280Deg_07152017.txt', 'SHG6_Background_275Deg_07152017.txt', 'SHG6_Signal_275Deg_07152017.txt', 'SHG6_Signal_270Deg_07152017.txt', 'SHG6_Background_270Deg_07152017.txt', 'SHG6_Background_265Deg_07152017.txt', 'SHG6_Signal_265Deg_07152017.txt', 'SHG6_Signal_260Deg_07152017.txt', 'SHG6_Background_260Deg_07152017.txt', 'SHG8_Signal_280Deg_071517.txt', 'SHG8_Background_280Deg_071517.txt', 'SHG8_Background_275Deg_071517.txt', 'SHG8_Signal_275Deg_071517.txt', 'SHG8_Signal_270Deg_071517.txt', 'SHG8_Background_270Deg_071517.txt', 'SHG8_Background_265Deg_071517.txt', 'SHG8_Signal_265Deg_071517.txt', 'SHG8_Signal_260Deg_071517.txt', 'SHG8_Background_260Deg_071517.txt', 'SHG9_Signal_280Deg_071517.txt', 'SHG9_Background_280Deg_071517.txt', 'SHG9_Background_275Deg_071517.txt', 'SHG9_Signal_275Deg_071517.txt', 'SHG9_Signal_270Deg_071517.txt', 'SHG9_Background_270Deg_071517.txt', 'SHG9_Background_265Deg_071517.txt', 'SHG9_Signal_265Deg_071517.txt',

In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [11]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,100):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
PDvalues = PDvalues.values.reshape(100,len(allFiles))
Background = np.array([PDvalues[:,1],PDvalues[:,2],PDvalues[:,5],PDvalues[:,6],PDvalues[:,9],\
                       PDvalues[:,11],PDvalues[:,12],PDvalues[:,15],PDvalues[:,16],PDvalues[:,19],\
                       PDvalues[:,21],PDvalues[:,22],PDvalues[:,25],PDvalues[:,26],PDvalues[:,29],\
                       PDvalues[:,31],PDvalues[:,32],PDvalues[:,35],PDvalues[:,36],PDvalues[:,39]])
Background_mean = np.mean(Background, axis = 1)
PMT_Raw = np.array([PDvalues[:,0],PDvalues[:,3],PDvalues[:,4],PDvalues[:,7],PDvalues[:,8],\
                    PDvalues[:,10],PDvalues[:,13],PDvalues[:,14],PDvalues[:,17],PDvalues[:,18],\
                    PDvalues[:,20],PDvalues[:,23],PDvalues[:,24],PDvalues[:,27],PDvalues[:,28],\
                    PDvalues[:,30],PDvalues[:,33],PDvalues[:,34],PDvalues[:,37],PDvalues[:,38]])
MeanPMT1 = np.mean(PMT_Raw,axis = 1)

PMT_Net = MeanPMT1 - Background_mean
MeanNET = np.reshape(PMT_Net,(4,5)) # As you add more files, change the 4 to the number of runs. 7 is b/c I took 7 different angles
MeanNET2 = np.mean(MeanNET,axis =0)
THzEnergy = np.array([480,450,390,310,235])
PMT_STD = np.std(MeanNET,axis = 0)
PMT_Err = PMT_STD/(4**0.5)
w = (PMT_Err/(MeanNET2*log(10)))**-2

print(Background_mean)
print(MeanPMT1)
print(MeanNET)
#print(PMT_)
#print(MeanNET2)
LogPMT = np.log10(np.absolute(MeanNET2))
LogTHz = np.log10(np.absolute(THzEnergy))

[-0.213218 -0.203608 -0.182045 -0.167559 -0.165341 -0.20669  -0.201066
 -0.178634 -0.165596 -0.162331 -0.2021   -0.195527 -0.177865 -0.164973
 -0.158883 -0.193835 -0.186189 -0.174739 -0.15746  -0.154295]
[-0.255404 -0.237056 -0.211195 -0.19151  -0.177099 -0.246076 -0.232554
 -0.208632 -0.184352 -0.170049 -0.24097  -0.22349  -0.199314 -0.175269
 -0.16549  -0.237186 -0.220926 -0.199057 -0.173668 -0.158505]
[[-0.042186 -0.033448 -0.02915  -0.023951 -0.011758]
 [-0.039386 -0.031488 -0.029998 -0.018756 -0.007718]
 [-0.03887  -0.027963 -0.021449 -0.010296 -0.006607]
 [-0.043351 -0.034737 -0.024318 -0.016208 -0.00421 ]]


In [12]:
Delta = sum(w)*sum(w*LogTHz**2) - (sum(w*LogTHz))**2
A = (sum(w*LogTHz**2)*sum(w*LogPMT) - sum(w*LogTHz)*sum(w*LogTHz*LogPMT))/Delta
B = (sum(w)*sum(w*LogTHz*LogPMT) - (sum(w*LogPMT)*sum(w*LogTHz)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogTHz)**2)**0.5)
errorA = (sum(w*LogTHz**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)

[ 10113.06759779   3307.50105245   1185.24143249    262.89702831
    163.38714081]
-7.40476957347
2.24081122107
0.474694446019
0.178344947375


In [13]:
plt.plot(LogTHz, A + B*LogTHz,'r',label = 'Fit: y=2.24x -7.40')
plt.plot(LogTHz, LogPMT,'b.', label = 'Data')
plt.grid(True)
plt.xlabel('Log(THz Energy)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('PMTSignalVsTHzEnergy_07152017.png')